In [88]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
from tqdm import tqdm

from uszipcode import SearchEngine, ComprehensiveZipcode

In [89]:
fires = pd.read_csv('20kFires.csv')
print(fires.head())

   Unnamed: 0  Unnamed: 0.1  FIRE_YEAR STAT_CAUSE_DESCR   LATITUDE  \
0           0             0       2010    Equipment Use  36.766944   
1           1             1       2010    Miscellaneous  36.776944   
2           2             2       2010    Miscellaneous  36.856111   
3           3             3       2010    Miscellaneous  36.818056   
4           4             4       2010    Miscellaneous  36.883056   

    LONGITUDE STATE  DISCOVERY_DATE  FIRE_SIZE        DATE     zipcode  \
0 -121.303056    CA       2455335.5       70.0  2010-05-19  95075-9701   
1 -121.311111    CA       2455355.5        0.5  2010-06-08       95023   
2 -121.381111    CA       2455359.5        0.1  2010-06-12       95023   
3 -121.391111    CA       2455361.5        0.1  2010-06-14       95023   
4 -121.561944    CA       2455388.5        1.0  2010-07-11  95045-9662   

                                             address  
0  Saddle Association Museum, Airline Highway, Tr...  
1  Pinnacle Hills Golf C

In [90]:
locator = geopy.Nominatim(user_agent='timothy', timeout=10)

#Apply reverse geoencoding
fires['LocationString'] = fires['LATITUDE'].map(str) + ',' + fires['LONGITUDE'].map(str)
print(fires.head())

   Unnamed: 0  Unnamed: 0.1  FIRE_YEAR STAT_CAUSE_DESCR   LATITUDE  \
0           0             0       2010    Equipment Use  36.766944   
1           1             1       2010    Miscellaneous  36.776944   
2           2             2       2010    Miscellaneous  36.856111   
3           3             3       2010    Miscellaneous  36.818056   
4           4             4       2010    Miscellaneous  36.883056   

    LONGITUDE STATE  DISCOVERY_DATE  FIRE_SIZE        DATE     zipcode  \
0 -121.303056    CA       2455335.5       70.0  2010-05-19  95075-9701   
1 -121.311111    CA       2455355.5        0.5  2010-06-08       95023   
2 -121.381111    CA       2455359.5        0.1  2010-06-12       95023   
3 -121.391111    CA       2455361.5        0.1  2010-06-14       95023   
4 -121.561944    CA       2455388.5        1.0  2010-07-11  95045-9662   

                                             address  \
0  Saddle Association Museum, Airline Highway, Tr...   
1  Pinnacle Hills Golf

In [92]:
from tqdm import trange


sz = len(fires)
resaddress = []

for i in trange(sz):
    coordinates = fires['LocationString'][i]
    location = locator.reverse(coordinates)
    resaddress.append(str(location))



  0%|          | 0/23035 [00:00<?, ?it/s]

  0%|          | 1/23035 [00:00<1:12:52,  5.27it/s]

  0%|          | 2/23035 [00:00<2:01:42,  3.15it/s]

  0%|          | 3/23035 [00:01<2:11:28,  2.92it/s]

  0%|          | 4/23035 [00:01<2:43:40,  2.35it/s]

  0%|          | 5/23035 [00:02<2:41:36,  2.38it/s]

  0%|          | 6/23035 [00:02<2:52:26,  2.23it/s]

  0%|          | 7/23035 [00:03<2:59:24,  2.14it/s]


KeyboardInterrupt: 

In [93]:
fires['address'] = resaddress

ValueError: Length of values (7) does not match length of index (23035)

In [101]:
zips = []
for i in range(sz):
    arr = fires['address'][i].split(',')
    zipc = arr[len(arr)-2]
    zipc = zipc.split('-')[0].strip()
    zipc = zipc.split(' ')
    zipc = zipc[len(zipc)-1]
#     print(zipc)
    zips.append(zipc)
#     print(zipc)

print(zips[0])
fires['zipcode'] = zips

95075


In [102]:
search = SearchEngine(
... simple_or_comprehensive=SearchEngine.SimpleOrComprehensiveArgEnum.comprehensive
)

In [119]:
def dealWithMissingValue():
    #Another option is to replace with the average of the entire test column of **the training dataset**
    return 0
#     return -9999

In [113]:
#features of interest
mp = {
    "population": [],
    "population_density": [],
    "population_by_race" : [],
    "land_area_in_sqmi": [],
    "water_area_in_sqmi": [],
    "housing_units": [],
    "occupied_housing_units": [],
    "median_home_value": [],
    "median_household_income": [],
    "sources_of_household_income____average_income_per_household_by_income_source": []
}

nons = []
nonsIdx = []

for i in trange(sz):
    info = search.by_zipcode(fires['zipcode'][i])
    if info is None:
        nons.append(i)
        continue
    info = info.to_dict()
#     for k in info:
#         print(k)
    for k in mp:
        if info[k] is None:
            nons.append(info)
            nonsIdx.append(i)
#             info[k] = dealWithMissingValue()
            #Then do the radius search thing and find a zipcode that has a value
        mp[k].append(info[k])
        
print(len(fires))
print(len(nons))
#approximatrely half will have missing values...



  0%|          | 0/23035 [00:00<?, ?it/s]

  0%|          | 8/23035 [00:00<04:52, 78.77it/s]

  0%|          | 89/23035 [00:00<03:32, 108.01it/s]

  1%|          | 186/23035 [00:00<02:35, 147.25it/s]

  1%|          | 273/23035 [00:00<01:56, 196.08it/s]

  2%|▏         | 346/23035 [00:00<01:30, 250.94it/s]

  2%|▏         | 413/23035 [00:00<01:13, 308.52it/s]

  2%|▏         | 480/23035 [00:00<01:01, 367.82it/s]

  2%|▏         | 545/23035 [00:00<00:56, 398.73it/s]

  3%|▎         | 606/23035 [00:00<00:52, 429.12it/s]

  3%|▎         | 680/23035 [00:01<00:45, 490.54it/s]

  3%|▎         | 758/23035 [00:01<00:43, 515.05it/s]


KeyboardInterrupt: 

In [118]:
print(nons[100])

{'zipcode': '95604', 'zipcode_type': 'PO BOX', 'major_city': 'Auburn', 'post_office_city': 'Auburn, CA', 'common_city_list': ['Auburn', 'Bowman'], 'county': 'Placer County', 'state': 'CA', 'lat': 39.24, 'lng': -120.07, 'timezone': 'America/Los_Angeles', 'radius_in_miles': 8.0, 'area_code_list': '530', 'population': None, 'population_density': None, 'land_area_in_sqmi': None, 'water_area_in_sqmi': None, 'housing_units': None, 'occupied_housing_units': None, 'median_home_value': None, 'median_household_income': None, 'bounds_west': -120.078544, 'bounds_east': -120.064082, 'bounds_north': 39.251787, 'bounds_south': 39.235853, 'polygon': [[-120.072798, 39.238265], [-120.07289, 39.238355], [-120.072923, 39.23838], [-120.072963, 39.238411], [-120.073019, 39.238447], [-120.073109, 39.238494], [-120.073216, 39.238544], [-120.073297, 39.23857], [-120.073384, 39.238587], [-120.073552, 39.238622], [-120.073486, 39.238754], [-120.073462, 39.23879], [-120.073442, 39.238821], [-120.073412, 39.238883